In [1]:
import Acqu as aq
import AcquDetector as aqdet
import Timepix
import ROOT
import numpy as np
import root_numpy
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Open defined file

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat' #Amorphous
#inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_34.dat' #Para(/Perp?)
#inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_50.dat' #Perp(/Para?)
aq.openFile(inFile)

Mk2 Data


In [6]:
#Set global variables before starting 
triggerTime  = []
scalerRepeat = 0
prevLong     = 0
pulseCount   = 0
EPICSN       = 0
EPICSEventA  = 0
EPICSEventB  = 0
prevTriggerTime = [0]

#Define Histograms and ranges
timeWindowMin  = -400000
timeWindowMax  = 400000

timeMax        = 200000000
nBins          = 300

timeWin        = 2000000
timeDiffDouble = Hist(nBins,-timeWin,timeWin,name='DoubleCoincidence')
timeDiff       = Hist(nBins,-timeWin,timeWin,name='Coincidence')

timeA          = Hist(40000,0,timeMax,name='TimeA')
timeB          = Hist(40000,0,timeMax,name='TimeB')

#h = Hist2D((timeWindowMax-timeWindowMin)/1000,timeWindowMin,timeWindowMax,40,0,40)
hA  = Hist((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,name='TimesA')
hA2  = Hist((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,name='TimesA2')
h2A = Hist2D((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,100,0,timeMax,name='TimesA2D')
hB  = Hist((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,name='TimesB')
h2B = Hist2D((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,100,0,timeMax,name='TimesB2D')

positionA = Hist2D(256,0,256,256,0,256,name='posA')
positionACoin = Hist2D(256,0,256,256,0,256,name='posACoin')

#Define function for matching trigger and timepix information
def TimepixTagger():
    global prevTriggerTime
    global triggerTime
    global scalerRepeat
    global prevLong
    global pulseCount
    global EPICSEventA
    global EPICSEventB
    global EPICSN
    
    # ADC value which increases on a pulser
    ADC104    = aq.adcArray[np.where(aq.adcArray['adc']==104)[0]]['val'][0]
    # ADC values which define the clock of the trigger
    shortTime = aq.adcArray[np.where(aq.adcArray['adc']==102)[0]]['val'][0]
    longTime  = aq.adcArray[np.where(aq.adcArray['adc']==103)[0]]['val'][0]
    # Calibration of the clock to roughly ns
    # Add this time to the list of times
    triggerTime += [8*(shortTime+65536*(longTime+65536*scalerRepeat))]
    
    # Included because there was some problem at the start of the file, probably a better way to do it
    if(len(triggerTime)<=1): return
    
    # If event contains epics information
    if(aq.epicsEvent==1):
        # Check if the epics information has been updated since the previous epics event
        if(EPICSEventA!=aq.getEpicsPV('PPOL:TIMEPIXA:EVENT') and EPICSEventB!=aq.getEpicsPV('PPOL:TIMEPIXB:EVENT') ):
            # Get the raw information out of the epics event
            EPICSEventA  = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
            EPICSEventB  = aq.getEpicsPV('PPOL:TIMEPIXB:EVENT')
            nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
            encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
            encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
            
            # Decode the information into a labelled vector. (Could probably swap to Kens pandas methods here)
            TimepixAData = Timepix.Decode(nHitsA,encodedA)
            TimepixBData = Timepix.Decode(nHitsB,encodedB)
            # Calibrate Timepix times
            nsTimeA = 16*TimepixAData[['ToA']].astype(long) - TimepixAData[['FToA']].astype(long)
            nsTimeB = 16*TimepixBData[['ToA']].astype(long) - TimepixBData[['FToA']].astype(long)
            # Determine cluster groupings
            diffA = np.concatenate([[0],np.diff(nsTimeA)])
            diffB = np.concatenate([[0],np.diff(nsTimeB)])            
            breakA = np.concatenate([[0],np.where(diffA>200)[0],[len(nsTimeA)]])
            breakB = np.concatenate([[0],np.where(diffB>200)[0],[len(nsTimeB)]])
            
            # Variables for the cluster only (Again Kens current approach is probably neater)
            timesA = []
            timesB = []
            lengA  = []
            lengB  = []
            posA   = []
            posB   = []
            if(len(nsTimeA)==0): breakA = []
            if(len(nsTimeB)==0): breakB = []
            
            # Clusters in detector A
            for ssi in zip(breakA[:-1],breakA[1:]):
                diffs = nsTimeA[ssi[0]:ssi[1]]-nsTimeA[ssi[0]] # Time differences 
                leng  = ssi[1]-ssi[0] # Cluster size
                lengA  += [leng]
                timesA += [nsTimeA[ssi[0]]] # Time of first hit
                posA   += [TimepixAData[['x','y']][ssi[0]]] # Position of first hit in cluster (Mean position would probably improve)
            
            
            # Clusters in detector B
            for ssi in zip(breakB[:-1],breakB[1:]):
                diffs = nsTimeB[ssi[0]:ssi[1]]-nsTimeB[ssi[0]]
                leng  = ssi[1]-ssi[0]
                lengB  += [leng]
                timesB += [nsTimeB[ssi[0]]]
                posB   += [TimepixBData[['x','y']][ssi[0]]]
            
            #Convert into numpy array for some reason
            timesA = np.array(timesA)
            timesB = np.array(timesB)
            posA = np.array(posA)
            posB = np.array(posB)
            
            # Fill raw positions of clusters
            positionA.fill_array(np.vstack([posA['x'],posA['y']]).T)
            
            # Fill raw cluster hit times
            timeA.fill_array(timesA)
            timeB.fill_array(timesB)
##                        
            EPICSN += 1
##            
            # If the trigger clock has looped add the loop length to the variables
            triggerTime2 = np.array(triggerTime,dtype=np.float)
            triggerWrap  = np.where(np.diff(triggerTime2)<0)[0]
            if(len(triggerWrap)>0):
                triggerTime2[triggerWrap[0]:] += 65536*65536*8
                
            # Determine which pulser trigger relates to the timepix event
            # This seems to get mixed up between the previous one and the one before that
            # Some are still wrong but enough right not to worry for now.
            timeSub = prevTriggerTime[-1]
            if((triggerTime2-timeSub)[-1]<1000000000 and len(prevTriggerTime)>1): timeSub = prevTriggerTime[-2]
            
            # Subtract that previous time
            triggerTime2 -= timeSub
            # Convert the clock times into 1.5625ns.
            # The differnce between the timepix and trigger clocks seems to be by a factor 1.015595 or there abouts
            triggerTime2 /= 1.015595*1.5625
            
            # Coincidence index list
            coinIdx = []
            
            # Loop through the timepix times finding which ones are in coincidence with trigger events
            for i, hit in enumerate(timesA):
                # Difference between the trigger times and the Timepix times
                tdiff = triggerTime2-hit
                # If there is a coincidence add the event index to the list
                # For some reason there are two peaks...
                # Could have a separate list of random events which could be scaled and subtracted like with the tagger.
                if (len(tdiff[np.abs(tdiff)<10000])>0 or len(tdiff[np.abs(tdiff-263000)<10000])>0):
                    coinIdx +=  [i]
                           
            # Make arrays just containing coincedent hits with Timepix A    
            coinHit  = timesA[coinIdx]
            coinPosA = posA[coinIdx]
            # Fill histogram with positions of coincident events in Timepix A
            positionACoin.fill_array(np.vstack([coinPosA['x'],coinPosA['y']]).T)
            
            
            # Repeat for Timepix B - Something is wrong with the timing in this detector
            # so the coincident peak is really broad, we should be able to match up events with some confidence though
            
            for hit in timesB:
                tdiff = triggerTime2-hit
                #print -nsTimeA+hit
                h2B.fill_array(np.vstack((tdiff,np.full(len(triggerTime2),hit))).T)
                hB.fill_array(tdiff)
                tpxDiff  = coinHit-hit
                tpxDiff2 = timesA-hit
                timeDiff.fill_array(tpxDiff2)
                timeDiffDouble.fill_array(tpxDiff)
                
            # NOT QUITE FINISHED
            # Things to do here -
            # Coincidence positions in Timepix B
            # Triple coincidence positions in both detectors (shouldn't be much different as thats how the trigger should have been made)
            # Look at the tagger hits and plot the coincident positions for different energy ranges
                
            # Update the trigger times for the next buffer
            # A bit more complex than ideal again because it might not be just the previous set of events.
            triggerfilter = prevTriggerTime[-1]
            if(triggerTime[0]>triggerfilter): triggerfilter += 65536*65536*8
            triggerTime     = triggerTime[np.where(triggerTime<=triggerfilter)[0][-1]:]
                
            prevTriggerTime = []
            
            
    if(aq.eventNo%10000==0):
        print("Events Processed: ",aq.eventNo)
    # Update the times that the trigger pulser was recorded.
    if(ADC104!=pulseCount):
        #print pulseCount, ADC104
        pulseCount  = ADC104        
        #print len(triggerTime)
        #print triggerTime[-1]-triggerTime[0]
        #print ' '
        prevTriggerTime += [triggerTime[-1]]
        #triggerTime = []

# Run the function - Pretty slow and could be optimised but fine for playing around with
aq.runFunction(TimepixTagger,0,5000000)



608
('len', 607)
582
('len', 581)
573
('len', 572)
582
('len', 581)
541
('len', 540)
598
('len', 597)
556
('len', 555)
('Events Processed: ', 10000)
542
('len', 541)
587
('len', 586)
571
('len', 570)
555
('len', 554)
565
('len', 564)
582
('len', 581)
('Events Processed: ', 20000)
530
('len', 529)
541
('len', 540)
562
('len', 561)
559
('len', 558)
619
('len', 618)
571
('len', 570)
564
('len', 563)
('Events Processed: ', 30000)
567
('len', 566)
555
('len', 554)
550
('len', 549)
589
('len', 588)
591
('len', 590)
537
('len', 536)


KeyboardInterrupt: 

In [5]:

ROOT.enableJSVis()
c41 = ROOT.TCanvas("my41","The Canvas Title",1000,600)
positionA.Draw("colz")
c41.Draw()
c42 = ROOT.TCanvas("my42","The Canvas Title",1000,600)
positionACoin.Draw("colz")
c42.Draw()

In [6]:

ROOT.enableJSVis()
c51 = ROOT.TCanvas("my51","The Canvas Title",1000,600)
positionA.RebinX(2)
positionA.RebinY(2)
positionA.Draw("colz")
c51.Draw()
c52 = ROOT.TCanvas("my52","The Canvas Title",1000,600)
positionACoin.RebinX(2)
positionACoin.RebinY(2)
positionACoin.Draw("colz")
c52.Draw()

In [7]:

ROOT.enableJSVis()
c3 = ROOT.TCanvas("my3","The Canvas Title",1000,600)
timeDiff.Draw("colz")
c3.Draw()
c31 = ROOT.TCanvas("my31","The Canvas Title",1000,600)
timeDiffDouble.Draw("colz")
c31.Draw()

ROOT.enableJSVis()
c21 = ROOT.TCanvas("my21","The Canvas Title",1200,600)
timeA.Draw("colz")
c21.Draw()
c22 = ROOT.TCanvas("my22","The Canvas Title",1200,600)
timeB.Draw("colz")
c22.Draw()

In [8]:
ROOT.enableJSVis()
c51 = ROOT.TCanvas("my51","The Canvas Title",1000,600)
hA.Draw("colz")
c51.Draw()
c50 = ROOT.TCanvas("my50","The Canvas Title",1000,600)
hA2.Draw("colz")
c50.Draw()
c52 = ROOT.TCanvas("my52","The Canvas Title",1000,600)
h2A.Draw("colz")
c52.Draw()
c53 = ROOT.TCanvas("my53","The Canvas Title",1000,600)
hB.Draw("colz")
c53.Draw()
c54 = ROOT.TCanvas("my54","The Canvas Title",1000,600)
h2B.Draw("colz")
c54.Draw()